#### Authentication

create gee non profit project

In [2]:
# import ee
# ee.Authenticate()
# ee.Initialize()

#4/1Ab_5qlnP5Q6eLQKUaOhmM-GKuDWSGDoOQxKuJcIpdt7YIoyh6zLow8qvREY


Successfully saved authorization token.


#### Read data

In [1]:
import ee
import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
ee.Initialize() #init instance

In [5]:
#define region of interest
roi = ee.Geometry.Polygon([[[-74.8333, 10.5000], #sw 10° 30′ N, 74° 50′ O → (-74.8333, 10.5000)
                            [-74.3333, 10.5000], #se 10° 30′ N, 74° 20′ O → (-74.3333, 10.5000)
                            [-74.3333, 11.2500], #ne 11° 15′ N, 74° 20′ O → (-74.3333, 11.2500)
                            [-74.3333, 11.2500]  #nw 11° 15′ N, 74° 50′ O → (-74.8333, 11.2500)
                            ]])

In [6]:
#sentinel 2
sentinel = (ee.ImageCollection('COPERNICUS/S2_SR') #deprecated
            .filterBounds(roi)
            .filterDate('2023-01-01', '2023-12-31')
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30)) #cloud %
            .select(['B2','B3','B4','B8']))  # b, g, r, nir


In [7]:
# Sentinel-2 Harmonized Surface Reflectance (Level-2A)
sentinel = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(roi)
            .filterDate('2023-01-01', '2023-12-31')
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
            .select(['B2','B3','B4','B8'])  # b, g, r, nir
            )

In [8]:
sentinel.size().getInfo()

138

In [9]:
dates = sentinel.aggregate_array('system:time_start').getInfo()

[datetime.fromtimestamp(t/1000).strftime('%Y-%m-%d') for t in dates][:5]

['2023-01-01', '2023-01-03', '2023-01-03', '2023-01-06', '2023-01-06']

define params e.g. turbidity

$t=\frac{B4 (\text{NIR})-B3 (\text{red})}{B4+B3}$

In [10]:
def add_turbidity(image): #relation r/b or tsm
    turb = image.normalizedDifference(['B4','B3']).rename('turbidity')
    return image.addBands(turb)

get images

In [11]:
base = sentinel.median() # ee.Image
turbidity = base.normalizedDifference(['B4','B3']) # (B4−B3)/(B4+B3)
turbidity = turbidity.rename('turbidity') # ee.Image

composite = base.addBands(turbidity).unmask(0) # ee.Image with 5 bands

In [12]:
type(composite)

ee.image.Image

In [13]:
patch_size, scale = 128, 10

In [14]:
proj = composite.projection().atScale(scale) # ee.Projection con CRS
reproj = composite.reproject(proj) #forces reprojection to 10 meters

#### Use

get region

In [15]:
lon, lat = -74.6, 11.0

half_m = (patch_size//2) * scale
meters_per_deg = 111320
half_deg = half_m / meters_per_deg

In [16]:
region = ee.Geometry.Rectangle([
    lon - half_deg, lat - half_deg,
    lon + half_deg, lat + half_deg
])

In [17]:
patch_feature = reproj.sampleRectangle(region=region,
                                       defaultValue=0
                                       )

In [18]:
patch_dict = patch_feature.toDictionary()
patch_data = patch_dict.getInfo()

to np

In [19]:
patch_data.keys()

dict_keys(['B2', 'B3', 'B4', 'B8', 'turbidity'])

In [25]:
bands = patch_data.keys()

arrays = [np.array(patch_data[b]) for b in bands]
tensor = np.stack(arrays, axis=0) #stack on axis 0 -> (5, 128, 128)

In [26]:
tensor.shape

(5, 129, 129)

In [29]:
np.save('batch.npy', tensor)